In [1]:
import numpy as np
import math
import pandas as pd

In [2]:
# Se representa de forma matricial cada uno de los coeficientes de las restricciones, incluyendo ambos las variables de holgura, exceso y 
# artificiales
# en el orden x1, x2, x3, x4, x5, R1, R2

restricciones = np.array([[2,1,-1,0,0,1,0], #R1
                         [1,-3,2,-1,0,0,1], #R2
                         [1,1,1,0,1,0,0]],  #x5
                         dtype=float)
# vector de soluciones
soluciones  = np.array([10,5,15] ,dtype=float)
# vector de la función objetivo, también en orden x1, x2, x3, x4, x5, R1, R2
# Para usar el método de big M, se le asignan a R1 y R2 valores de 100
z = np.array([0,0,0,0,0,-1,-1], dtype=float)

# Agregar columna de soluciones a la matriz
tabla = np.hstack((restricciones, soluciones.reshape(-1, 1)))
#Añadir la fila de z
tabla = np.vstack((tabla, np.append(z, 0)))

columnas = ['x1', 'x2', 'x3', 'x4', 'x5', 'R1', 'R2', 'solucion']
filas = ['R1', 'R2', 'x5', 'Z']

# Crear y mostrar el DataFrame
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)




     x1   x2   x3   x4   x5   R1   R2  solucion
R1  2.0  1.0 -1.0  0.0  0.0  1.0  0.0      10.0
R2  1.0 -3.0  2.0 -1.0  0.0  0.0  1.0       5.0
x5  1.0  1.0  1.0  0.0  1.0  0.0  0.0      15.0
Z   0.0  0.0  0.0  0.0  0.0 -1.0 -1.0       0.0


In [3]:
z_vieja = tabla[-1, :].copy()
z_nueva = z_vieja -tabla[0, :] - tabla[1, :]
tabla[-1, :] = z_nueva

df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)


     x1   x2   x3   x4   x5   R1   R2  solucion
R1  2.0  1.0 -1.0  0.0  0.0  1.0  0.0      10.0
R2  1.0 -3.0  2.0 -1.0  0.0  0.0  1.0       5.0
x5  1.0  1.0  1.0  0.0  1.0  0.0  0.0      15.0
Z  -3.0  2.0 -1.0  1.0  0.0 -2.0 -2.0     -15.0


In [4]:
def revisar_minimizacion(tabla, num_fase):
    if num_fase==1:
        z_solution = tabla[-1, -1]
        if abs(z_solution) < 1e-6: 
            no_hay_artificiales = False
            for var in filas:
                if var in ['R1', 'R2']:
                    no_hay_artificiales = True
            return no_hay_artificiales
        else:
            return True
    elif num_fase ==2:
        return any(var <0 for var in tabla[-1,:])


def pivote(tabla, num_fila_piv, num_col_piv):
    nueva_tabla = tabla.copy()

    #Se toma solo la fila pivote, se edita y se devuelve a la tabla
    fila_pivote = nueva_tabla[num_fila_piv, :].copy()
    pivote_elemento = fila_pivote[num_col_piv]
    fila_pivote = fila_pivote / pivote_elemento
    nueva_tabla[num_fila_piv, :] = fila_pivote

    for i in range(nueva_tabla.shape[0]):
        if i != num_fila_piv:
            fila_actual = nueva_tabla[i, :].copy()
            coef = fila_actual[num_col_piv]
            nueva_tabla[i, :] = fila_actual - (coef * fila_pivote)

    return nueva_tabla




def solver_minimizacion(tabla, filas, columnas, num_fase):
    iteracion = 1
    
    while revisar_minimizacion(tabla, num_fase):
        print (f"Iteración {iteracion}:")
        columna = np.argmin(tabla[-1,:-1])
        print (f"Columna pivote:  {columna}. Entra la variable {columnas[columna]} como básica")       
        #Encontrar la fila pivote
        num_fila = 0
        ratios = []
        for i in range(tabla[:-1].shape[0]):
            if tabla[i, columna] > 0:
                posible = tabla[i, -1] / tabla[i, columna]
                ratios.append(posible)
            else:
                ratios.append(np.inf)

        
        num_fila = np.argmin(ratios)
        print (f"Fila pivote:  {num_fila}. Sale la variable {filas[num_fila]} como básica.") 
        filas[num_fila] = columnas[columna]
        tabla = pivote(tabla, num_fila, columna)
        print("tabla actual:")
        print (pd.DataFrame(tabla, columns=columnas, index=filas))
        iteracion+=1
    print("Solución encontrada!!: ")
    return tabla


columnas = ['x1', 'x2', 'x3', 'x4', 'x5', 'R1', 'R2', 'solucion']
filas = ['R1', 'R2', 'x5','Z']
tabla = solver_minimizacion(tabla, filas, columnas,num_fase=1)

df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

        

Iteración 1:
Columna pivote:  0. Entra la variable x1 como básica
Fila pivote:  0. Sale la variable R1 como básica.
tabla actual:
     x1   x2   x3   x4   x5   R1   R2  solucion
x1  1.0  0.5 -0.5  0.0  0.0  0.5  0.0       5.0
R2  0.0 -3.5  2.5 -1.0  0.0 -0.5  1.0       0.0
x5  0.0  0.5  1.5  0.0  1.0 -0.5  0.0      10.0
Z   0.0  3.5 -2.5  1.0  0.0 -0.5 -2.0       0.0
Iteración 2:
Columna pivote:  2. Entra la variable x3 como básica
Fila pivote:  1. Sale la variable R2 como básica.
tabla actual:
     x1   x2   x3   x4   x5   R1   R2  solucion
x1  1.0 -0.2  0.0 -0.2  0.0  0.4  0.2       5.0
x3  0.0 -1.4  1.0 -0.4  0.0 -0.2  0.4       0.0
x5  0.0  2.6  0.0  0.6  1.0 -0.2 -0.6      10.0
Z   0.0  0.0  0.0  0.0  0.0 -1.0 -1.0       0.0
Solución encontrada!!: 
     x1   x2   x3   x4   x5   R1   R2  solucion
x1  1.0 -0.2  0.0 -0.2  0.0  0.4  0.2       5.0
x3  0.0 -1.4  1.0 -0.4  0.0 -0.2  0.4       0.0
x5  0.0  2.6  0.0  0.6  1.0 -0.2 -0.6      10.0
Z   0.0  0.0  0.0  0.0  0.0 -1.0 -1.0       

Ahora se eliminan las columnas de las variables artificiales.



In [5]:

df = df.drop(columns=['R1', 'R2'])
tabla = df.to_numpy()
#Se reescriben las columnas para que ya no tengan R1 ni R2
columnas = ['x1', 'x2', 'x3', 'x4', 'x5', 'solucion']

#Se reescribe la fila objetivo con las restricciones originales y la solución actual en 0
tabla[-1, :] = [-5, 4, -3, 0, 0,0]
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

     x1   x2   x3   x4   x5  solucion
x1  1.0 -0.2  0.0 -0.2  0.0       5.0
x3  0.0 -1.4  1.0 -0.4  0.0       0.0
x5  0.0  2.6  0.0  0.6  1.0      10.0
Z  -5.0  4.0 -3.0  0.0  0.0       0.0


Se encontró una solución donde
$$ x_1 - \frac{1}{5}x_2 - \frac{1}{5}x_4 = 5$$
$$ -\frac{7}{5}x_2 +x_3 - \frac{2}{5}x_4 = 0$$
$$ \frac{13}{5}x_2 + \frac{3}{5}x_4 + x_5 = 10$$

Como las variables básicas son x_1, x_3 y x_5, se reescriben las 3 ecuaciones para despejar esas 3 variables:
$$ x_1 = 5 +\frac{1}{5}x_2+ \frac{1}{5}x_4$$
$$ x_3 = \frac{7}{5}x_2 + \frac{2}{5}x_4$$
$$ x_5 = 10 - \frac{13}{5}x_2 - \frac{3}{5}x_4 $$


y se reescribe la fila de Z con las ecuaciones obtenidas previamente:

$$Z = 5x_1 -4x_2 +3x_3 +0x_4 +0x_5$$

$$Z = 5(5 +\frac{1}{5}x_2+\frac{1}{5}x_4) -4x_2 +3(\frac{7}{5}x_2 + \frac{2}{5}x_4)$$

$$ Z = 25 + x_2 +x_4 -4x_2 + \frac{21}{5}x_2 +  \frac{6}{5}x_4$$

$$ Z = 25 + \frac{6}{5}x_2 + \frac{11}{5}x_4 $$

Finalmente, se dejan todas las variables del mismo lado:

$$ Z - \frac{6}{5}x_2 - \frac{11}{5}x_4 = 25$$

Entonces se reescribe la tabla de simplex con la nueva fila para Z:
$$[0,-\frac{6}{5}x_2,0,-\frac{11}{5}x_4,0,25]$$

In [6]:
tabla[-1,:] = np.array([0,-6/5,0,-11/5,0,25],dtype=float)
df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

     x1   x2   x3   x4   x5  solucion
x1  1.0 -0.2  0.0 -0.2  0.0       5.0
x3  0.0 -1.4  1.0 -0.4  0.0       0.0
x5  0.0  2.6  0.0  0.6  1.0      10.0
Z   0.0 -1.2  0.0 -2.2  0.0      25.0


Ahora se resuelve de nuevo

In [7]:
tabla = solver_minimizacion(tabla, filas, columnas,num_fase=2)

df = pd.DataFrame(tabla, columns=columnas, index=filas)
print(df)

Iteración 1:
Columna pivote:  3. Entra la variable x4 como básica
Fila pivote:  2. Sale la variable x5 como básica.
tabla actual:
     x1        x2   x3   x4        x5   solucion
x1  1.0  0.666667  0.0  0.0  0.333333   8.333333
x3  0.0  0.333333  1.0  0.0  0.666667   6.666667
x4  0.0  4.333333  0.0  1.0  1.666667  16.666667
Z   0.0  8.333333  0.0  0.0  3.666667  61.666667
Solución encontrada!!: 
     x1        x2   x3   x4        x5   solucion
x1  1.0  0.666667  0.0  0.0  0.333333   8.333333
x3  0.0  0.333333  1.0  0.0  0.666667   6.666667
x4  0.0  4.333333  0.0  1.0  1.666667  16.666667
Z   0.0  8.333333  0.0  0.0  3.666667  61.666667
